In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

In [2]:
df = pd.read_csv('final_database.csv')
df = df.drop(df.columns[0], axis=1)

In [3]:
df.head()

,cuisine,recipe_ingredients,recipe_name
0,african,2 tbsp vegetable oil ½ tsp cumin seeds ½ tsp f...,Bunny chow
1,african,"1 tbsp olive or vegetable oil 2 large onions, ...",Jollof rice with fried plantains
2,african,400ml/14fl oz passata 3 tbsp tomato purée 2 fr...,Jollof rice
3,african,4g smoked paprika 2g cayenne pepper 6g ginger...,Suya fillet burger with sweet potato cubes and...
4,african,300g/10½oz basmati rice 1 tbsp vegetable oil 8...,Jollof rice with chicken


In [18]:
import nltk
from nltk.corpus import stopwords
import string
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import numpy as np
np.random.seed(0)

In [5]:
ingredients = df['recipe_ingredients'].tolist()

In [6]:
cuisine = df['cuisine'].tolist()

In [7]:
import re
stopwords_list = stopwords.words('english')+ list(string.punctuation)
stopwords_list+= ["''", '""', '...', '``']
stopwords_list+= ['tsp','tbsp',
             'tablespoon', 'teaspoon', 'tablespoons', 'teaspoons', 
             'large', 'cup', 'ounces', 'pound', 'oz', 'slice','sliced', 
             'cup', 'cups', 'ounce', 'ounces', 'chopped', 'finely', 'cut',
            'thinly', 'pounds', 'lb', 'lbs', 'g','oz', 'small', 'large']

#This function returns a PARAGRAPH of words that have stopwords and numbers removed
def process_recipe(recipe):
    tokens = nltk.word_tokenize(recipe)
    stopwords_removed = ' '.join([token.lower() for token in tokens if token not in stopwords_list])
    pattern= r"[a-z]+"
    regex_tokens = re.findall(pattern, stopwords_removed)
    return ' '.join(regex_tokens)

In [8]:
processed_ingredients = list(map(process_recipe, ingredients))

In [9]:
processed_ingredients

['vegetable oil cumin seeds fennel seeds cm in piece cinnamon stick green cardamom pods star anise bay leaf onion south african curry powder tomatoes kg lb oz boneless leg lamb cm in dices fresh ginger garlic curry leaves potatoes cubes size meat salt coriander leaves lime juice loaves crusty white bread unsliced across half middle crumbs removed coriander cress sprigs garnish',
 'olive vegetable oil onions x g oz cans plum tomatoes red pepper diced tomato pur e cayenne pepper chilli powder curry powder bay leaf sprig fresh thyme stock cube g oz long grain basmati rice salt freshly ground black pepper plantains slices deep fried green leaf salad',
 'ml fl passata tomato pur e fresh red scotch bonnet chillies seeds removed onions red peppers seeds removed roughly garlic cloves peeled fresh rosemary leaves fresh thyme leaves ground coriander sweet smoked paprika ml fl olive oil g oz cherry tomatoes halved ml pints chicken stock vegetable stock bay leaves g lb oz long grain rice rinsed su

In [10]:
ingredients_df = pd.DataFrame(processed_ingredients)
cuisines_df = pd.DataFrame(cuisine)


In [11]:
model_df = pd.concat([ingredients_df, cuisines_df], axis=1, sort=False)

In [12]:
model_df.columns = ['ingredients','cuisine']

In [21]:
model_df.head()

,ingredients,cuisine
0,vegetable oil cumin seeds fennel seeds cm in p...,african
1,olive vegetable oil onions x g oz cans plum to...,african
2,ml fl passata tomato pur e fresh red scotch bo...,african
3,g smoked paprika g cayenne pepper g ginger pow...,african
4,g oz basmati rice vegetable oil boneless skinl...,african


In [22]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(model_df.ingredients, model_df.cuisine, train_size = .8)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [23]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(model_df['ingredients'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(X_train)
xtest_count =  count_vect.transform(X_test)

In [26]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(model_df['ingredients'])
xtrain_tfidf =  tfidf_vect.transform(X_train)
xtest_tfidf =  tfidf_vect.transform(X_test)


In [47]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), max_features=5000)
tfidf_vect_ngram.fit(model_df['ingredients'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)


In [48]:
nb_classifier = MultinomialNB()
rf_classifier = RandomForestClassifier(n_estimators=100)

In [49]:
nb_classifier.fit(xtrain_tfidf_ngram, y_train)
nb_preds = nb_classifier.predict(xtest_tfidf_ngram)

In [50]:
nb_train_score = accuracy_score(y_test, nb_preds)

In [52]:
nb_train_score

0.5529411764705883

In [53]:
rf_classifier.fit(xtrain_tfidf_ngram, y_train)
rf_preds = rf_classifier.predict(xtest_tfidf_ngram)
rf_train_score = accuracy_score(y_test, rf_preds)
rf_train_score

0.6617647058823529

In [54]:
from sklearn.neighbors import KNeighborsClassifier as KNC
knn_classifier= KNC()
knn_classifier.fit(xtrain_tfidf_ngram, y_train)
knn_test_preds= knn_classifier.predict(xtest_tfidf_ngram)
knn_train_score = accuracy_score(y_test, knn_test_preds)
knn_train_score

0.46176470588235297